In [1]:
import sys

sys.path.append('../GraphStructureLearning')

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import mat73

from torch_geometric.data import Data as gData
from torch_geometric.utils import to_networkx, to_undirected
from torch_geometric.nn import MessagePassing

import networkx as nx
import matplotlib.pyplot as plt

from torch_geometric_temporal.nn.recurrent import DCRNN

# GTS

In [3]:
from models.GTS.gts_graph_learning import GTS_Graph_Learning
from models.GTS.gts_forecasting_module import GTS_Forecasting_Module
from models.GTS.DCRNN import DCRNN
from utils.utils import build_edge_idx

from glob import glob
import yaml
from easydict import EasyDict as edict

In [4]:
config_file = glob('./config/GTS/*.yaml')[0]
config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))

In [5]:
node_feas = torch.rand(config.nodes_num, config.node_features, 1000)
edge_index = build_edge_idx(num_nodes=config.nodes_num)

In [6]:
from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader = ChickenpoxDatasetLoader()

dataset = loader.get_dataset()

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

In [36]:
for time, snapshot in enumerate(train_dataset):
    pass

In [37]:
time

102

In [38]:
snapshot

Data(x=[20, 4], edge_index=[2, 102], edge_attr=[102], y=[20])

In [5]:
edge_index.shape

torch.Size([2, 870])

In [6]:
node_feas.shape

torch.Size([30, 1, 1000])

In [7]:
gl = GTS_Graph_Learning(config)

In [8]:
adj = gl(node_feas, edge_index)

In [9]:
z_1 = F.gumbel_softmax(adj, tau=0.3, hard=True)
z_1 = torch.transpose(z_1, 0, 1)

In [10]:
edge_ = []

for ii, rel in enumerate(z_1[0]):
    if bool(rel):
        edge_.append(edge_index[:, ii])

adj_matrix = torch.stack(edge_, dim=-1)

In [12]:
config.num_layer=2

In [13]:
config.embedding_dim = 13
config.in_channels = 1

In [14]:
encoder_dcrnn = DCRNN(config)
decoder_dcrnn = DCRNN(config)

In [15]:
inputs = node_feas.reshape(-1,30,1,100)
inputs.shape
# batch, node_num, node_feature, total time step

torch.Size([10, 30, 1, 100])

In [29]:
encoder_hidden_state = encoder_dcrnn(inputs[0,:,:,0], adj_matrix)

In [30]:
encoder_hidden_state_2 = encoder_dcrnn(inputs[0,:,:,1], adj_matrix, hidden_state=encoder_hidden_state)

In [31]:
decoder_hidden_state = decoder_dcrnn(inputs[0,:,:,3], adj_matrix, hidden_state=encoder_hidden_state_2)

In [32]:
decoder_hidden_state[-1].shape

torch.Size([30, 64])

In [25]:
forecasting = GTS_Forecasting_Module(config)

In [26]:
encoder_inputs = torch.rand(config.nodes_num, config.node_features, 70)
decoder_inputs = torch.rand(config.nodes_num, config.node_features, 30)

In [27]:
forecasting(encoder_inputs, decoder_inputs, adj_matrix)

torch.Size([30, 1, 70])


RuntimeError: Tensors must have same number of dimensions: got 3 and 2